In [ ]:
import time
import numpy as np
import pandas as pd

import Case
from imp import reload
reload(Case)

In [ ]:
mimic_annoted = Case.annotation(mimic, 'mimic')

analysis_cohort_for_mimic = []

idx = mimic_annoted[(mimic_annoted['ABPd']==75.00)|(mimic_annoted['ABPs']==125.00)].index

mimic_annoted['MAP_fillna'].loc[idx]=1
mimic_annoted['ABPd_fillna'].loc[idx]=1
mimic_annoted['ABPs_fillna'].loc[idx]=1

for stayid in mimic_annoted.stay_id.unique():

    interest = mimic_annoted[mimic_annoted['stay_id']==stayid].head(9)
    lactic = mimic_annoted[mimic_annoted['stay_id']==stayid].head(2)


    if all(interest['Annotation']=='no_circ'):
        analysis_cohort_for_mimic.append(stayid)
    
    if any(lactic['Lactate']==1.1):

        stay_df = mimic_annoted[mimic_annoted['stay_id']==stayid]
        lact_idx = stay_df[stay_df['Lactate']==1.1].index
        mimic_annoted['Lactate_fillna'].loc[lact_idx]=1

print('num of retain stay id: ', len(analysis_cohort_for_mimic))

check_mimic = mimic_annoted[mimic_annoted['stay_id'].isin(analysis_cohort_for_mimic)].reset_index(drop=True)
mimic_labeled = optimized_shock_labeler(check_mimic, 'mimic')


eicu_annoted = Case.annotation(eicu, 'eicu')

analysis_cohort_for_eicu = []

idx = eicu_annoted[(eicu_annoted['ABPd']==75.00)|(eicu_annoted['ABPs']==125.00)].index

eicu_annoted['MAP_fillna'].loc[idx]=1
eicu_annoted['ABPd_fillna'].loc[idx]=1
eicu_annoted['ABPs_fillna'].loc[idx]=1

for stayid in eicu_annoted.patientunitstayid.unique():

    interest = eicu_annoted[eicu_annoted['patientunitstayid']==stayid].head(9)
    lactic = eicu_annoted[eicu_annoted['patientunitstayid']==stayid].head(2)


    if all(interest['Annotation']=='no_circ'):
        analysis_cohort_for_eicu.append(stayid)
    
    if any(lactic['Lactate']==1.1):

        stay_df = eicu_annoted[eicu_annoted['patientunitstayid']==stayid]
        lact_idx = stay_df[stay_df['Lactate']==1.1].index
        eicu_annoted['Lactate_fillna'].loc[lact_idx]=1

print('num of retain stay id: ', len(analysis_cohort_for_eicu))

check_eicu = eicu_annoted[eicu_annoted['patientunitstayid'].isin(analysis_cohort_for_eicu)].reset_index(drop=True)
eicu_labeled = optimized_shock_labeler(check_eicu, 'eicu')

In [ ]:
# mimic_labeled.drop('Unnamed: 0', axis = 1).to_csv('check_point_mimic_labeled.csv.gz', compression = 'gzip', index=False)
# eicu_labeled.drop('Unnamed: 0', axis = 1).to_csv('check_point_eicu_labeled.csv.gz', compression = 'gzip', index=False)

In [ ]:
# mimic_labeled = pd.read_csv('check_point_mimic_labeled.csv.gz', compression = 'gzip')
# eicu_labeled = pd.read_csv('check_point_eicu_labeled.csv.gz', compression = 'gzip')

In [ ]:
invalid_columns_mimic, _ = Case.find_invalid_columns(mimic_labeled)
invalid_columns_eicu, _ = Case.find_invalid_columns(eicu_labeled)

m_labeled = Case.replace_inf_with_previous(mimic_labeled, invalid_columns_mimic)
e_labeled = Case.replace_inf_with_previous(eicu_labeled, invalid_columns_eicu)

In [ ]:
m_case1_case2, m_case3_case4 = Case.Case_definetion(m_labeled, 'mimic')
e_case1_case2, e_case3_case4 = Case.Case_definetion(e_labeled, 'eicu')

In [ ]:
case1_stay_ids = []
for stay_id, group in m_case1_case2.groupby('stay_id'):
    if all(group['Case'] == 1):
        case1_stay_ids.append(stay_id)

m_case1_case2_filtered = m_case1_case2[~(m_case1_case2['stay_id'].isin(case1_stay_ids))].reset_index(drop=True)

case1_stay_ids = []
for stay_id, group in e_case1_case2.groupby('patientunitstayid'):
    if all(group['Case'] == 1):
        case1_stay_ids.append(stay_id)

e_case1_case2_filtered = e_case1_case2[~(e_case1_case2['patientunitstayid'].isin(case1_stay_ids))].reset_index(drop=True)

In [ ]:
m_case1_case2['INDEX'] = 'CASE1_CASE2_DF'
m_case3_case4['INDEX'] = 'CASE3_CASE4_DF'

e_case1_case2['INDEX'] = 'CASE1_CASE2_DF'
e_case3_case4['INDEX'] = 'CASE3_CASE4_DF'

In [ ]:
mimic_analysis = pd.concat([m_case1_case2, m_case3_case4], axis = 0).reset_index(drop=True)
eicu_analysis = pd.concat([e_case1_case2, e_case3_case4], axis = 0).reset_index(drop=True)

In [ ]:
from tqdm import tqdm
def lactate_up(df, mode):
    targ = df.copy()
    
    if mode == 'mimic':
        stay_id_id = 'stay_id'
    elif mode == 'eicu':
        stay_id_id = 'patientunitstayid'
        
    targ['lactate_up'] = np.nan
        
    for stayid in tqdm(targ[stay_id_id].unique()):
        
        stay_trajectory = targ[targ[stay_id_id]==stayid]

        true_measurements_lactate = stay_trajectory['Lactate'].unique()
        true_measurements_lactate_index = []
        
        for value in true_measurements_lactate:
            first_occurrence_index = stay_trajectory[stay_trajectory['Lactate'] == value].index[0]
            true_measurements_lactate_index.append(first_occurrence_index)
        
        #init
        
        anchor_lactate = true_measurements_lactate[0]
        count_change = 0
        
        # true_measurements_lactate_index = [0, 1, 2, 3]
        # true_measurements_lactate = [(1.2 masking), 1.2, 2.0, 1.4, 0.7]
        
        lactate_count_dict = {element: index+1 for index, element in enumerate(true_measurements_lactate)}
        
        for idx, row in stay_trajectory.iterrows():
            #LACTATE COND
            current_lactate = row['Lactate']
            
            if lactate_count_dict[current_lactate] - count_change == 1:
                lactate_decrease = current_lactate <= anchor_lactate
            
            elif current_lactate != anchor_lactate:
                count_change += 1
                lactate_decrease = current_lactate <= anchor_lactate #바뀌는 순간은 그대로 계산해도 됨
                for key, value in lactate_count_dict.items():
                    if value == count_change:
                        anchor_lactate = key
            
            # Annotation
            
            if lactate_decrease:
                targ.at[idx, 'lactate_up'] = 0
            else :
                targ.at[idx, 'lactate_up'] = 1

    return targ


mimic_analysis_save = lactate_up(mimic_analysis, 'mimic')
eicu_analysis_save = lactate_up(eicu_analysis, 'eicu')

In [ ]:
mimic_analysis_save.to_csv('mimic_analysis.csv.gz', compression = 'gzip', index=False)
eicu_analysis_save.to_csv('eicu_analysis.csv.gz', compression = 'gzip', index=False)

In [2]:
import pandas as pd
mimic_data_dir = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/mimic_analysis.csv.gz'
mimic = pd.read_csv(mimic_data_dir, compression='gzip')

C:\Users\DAHS\AppData\Local\Temp\ipykernel_44536\3158904586.py:3: DtypeWarning: Columns (225) have mixed types. Specify dtype option on import or set low_memory=False.
  mimic = pd.read_csv(mimic_data_dir, compression='gzip')


In [15]:
mimic[mimic['stay_id']==mimic.stay_id.unique()[6]][['Time_since_ICU_admission', 'MAP', 'MAP_fillna', 'Lactate', 'Lactate_fillna', 'lactate_up', 'vasoactive/inotropic', 'Annotation', 'progress', 'Case']].iloc[40:60]

,Time_since_ICU_admission,MAP,MAP_fillna,Lactate,Lactate_fillna,lactate_up,vasoactive/inotropic,Annotation,progress,Case
724,40,91.666667,1,1.1,1,0.0,0,no_circ,0,1.0
725,41,91.666667,1,1.1,1,0.0,0,no_circ,0,1.0
726,42,91.666667,1,1.1,1,0.0,0,no_circ,0,1.0
727,43,91.666667,1,1.1,1,0.0,0,no_circ,0,1.0
728,44,91.666667,1,1.1,1,0.0,0,no_circ,0,2.0
729,45,91.666667,1,1.1,1,0.0,0,no_circ,0,2.0
730,46,91.666667,1,1.0,0,0.0,0,no_circ,0,2.0
731,47,91.666667,1,1.0,0,0.0,0,no_circ,0,2.0
732,48,91.666667,1,1.0,0,0.0,0,no_circ,0,2.0
733,49,91.666667,1,1.0,0,0.0,0,no_circ,0,2.0


In [16]:
len(mimic)

566610

In [17]:
mimic.stay_id.nunique()

3291

In [18]:
mimic.subject_id.nunique()

3218

In [19]:
import pandas as pd
eicu_data_dir = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/eicu_analysis.csv.gz'
eicu = pd.read_csv(eicu_data_dir, compression='gzip')

C:\Users\DAHS\AppData\Local\Temp\ipykernel_44536\3589023738.py:3: DtypeWarning: Columns (224) have mixed types. Specify dtype option on import or set low_memory=False.
  eicu = pd.read_csv(eicu_data_dir, compression='gzip')


In [21]:
eicu.patientunitstayid.nunique()

5563

In [22]:
eicu.uniquepid.nunique()

5397

In [23]:
len(eicu)

958448